In [1]:
import os 
import tensorflow as tf

2025-01-27 16:43:26.635997: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 16:43:26.701383: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 16:43:27.908864: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%cd /home/lab/Workspace/e2e-mlflow-dvc

/home/lab/Workspace/e2e-mlflow-dvc


In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/diazmanne/e2e-mlflow-dvc.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="diazmanne"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f4e87115c3d98a2ef151c0469443be9738fa35ff"

In [4]:
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict



@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    test_data: Path  # Added test_data here to match the code usage
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

# Validation function
def validate_evaluation_config(evaluation_config: EvaluationConfig):
    print(f"Model Path: {evaluation_config.path_of_model}")
    print(f"Training Data Path: {evaluation_config.training_data}")
    print(f"Test Data Path: {evaluation_config.test_data}")
    print(f"MLflow URI: {evaluation_config.mlflow_uri}")
    print(f"Image Size: {evaluation_config.params_image_size}")
    print(f"Batch Size: {evaluation_config.params_batch_size}")
    print(f"All Params: {evaluation_config.all_params}")

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config["artifacts_root"]])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config["evaluation"]["trained_model_path"]),
            training_data=Path(self.config["evaluation"]["training_data_path"]),
            test_data=Path(self.config["evaluation"]["test_data_path"]),  # Ensure test_data is defined
            mlflow_uri=self.config["mlflow_uri"],
            all_params=self.params,
            params_image_size=self.params["IMAGE_SIZE"],
            params_batch_size=self.params["BATCH_SIZE"]
        )
        return eval_config

# Instantiate ConfigurationManager and evaluate
config_manager = ConfigurationManager()
evaluation_config = config_manager.get_evaluation_config()

# Validate and debug
validate_evaluation_config(evaluation_config)

[2025-01-27 16:43:31,965: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-01-27 16:43:31,969: INFO: common: YAML file loaded successfully: params.yaml]
[2025-01-27 16:43:31,971: INFO: common: Created directory at: artifacts]
Model Path: artifacts/training/trained_model.keras
Training Data Path: data/train
Test Data Path: data/test
MLflow URI: https:/dagshub.com/diazmanne/e2e-mlflow-dvc.mlflow
Image Size: [224, 224, 3]
Batch Size: 16
All Params: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': PosixPath('imagenet'), 'LEARNING_RATE': 0.01}


In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import logging
from cnnClassifier.utils.common import save_json

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None
        logging.info("Evaluation instance created successfully.")

    def _valid_generator(self):
        try:
            logging.info("Initializing the validation data generator.")
            
            datagenerator_kwargs = dict(
                rescale=1.0 / 255,
                validation_split=0.30  # 30% for validation
            )

            dataflow_kwargs = dict(
                target_size=self.config.params_image_size[:-1],  # Exclude channels
                batch_size=self.config.params_batch_size,
                interpolation="bilinear"
            )

            valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )

            self.valid_generator = valid_datagenerator.flow_from_directory(
                directory=self.config.training_data,
                subset="validation",
                shuffle=False,
                **dataflow_kwargs
            )
            logging.info("Validation data generator initialized successfully.")

        except Exception as e:
            logging.error(f"Error initializing validation generator: {e}")
            raise e

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        try:
            logging.info(f"Loading model from: {path}")
            model = tf.keras.models.load_model(path)
            logging.info("Model loaded successfully.")
            return model
        except Exception as e:
            logging.error(f"Error loading model: {e}")
            raise e

    def evaluation(self):
        try:
            logging.info("Starting model evaluation.")
            self.model = self.load_model(self.config.path_of_model)
            self._valid_generator()
            
            if self.valid_generator is None:
                raise ValueError("Validation generator not initialized.")
            
            self.score = self.model.evaluate(self.valid_generator)
            logging.info(f"Evaluation completed. Loss: {self.score[0]}, Accuracy: {self.score[1]}")
            self.save_score()

        except Exception as e:
            logging.error(f"Error during evaluation: {e}")
            raise e

    def save_score(self):
        try:
            logging.info("Saving evaluation scores.")
            scores = {"loss": self.score[0], "accuracy": self.score[1]}
            save_json(path=Path("scores.json"), data=scores)
            logging.info(f"Scores saved successfully: {scores}")

        except Exception as e:
            logging.error(f"Error saving scores: {e}")
            raise e

    def log_into_mlflow(self):
        try:
            logging.info("Logging metrics and model to MLflow.")
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                    {"loss": self.score[0], "accuracy": self.score[1]}
                )

                if tracking_url_type_store != "file":
                    mlflow.keras.log_model(
                        self.model, 
                        "model", 
                        registered_model_name="VGG16Model"
                    )
                else:
                    mlflow.keras.log_model(self.model, "model")

            logging.info("Logging to MLflow completed successfully.")

        except Exception as e:
            logging.error(f"Error during MLflow logging: {e}")
            raise e



In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-01-27 16:43:32,347: INFO: common: YAML file loaded successfully: config/config.yaml]
[2025-01-27 16:43:32,353: INFO: common: YAML file loaded successfully: params.yaml]
[2025-01-27 16:43:32,355: INFO: common: Created directory at: artifacts]
[2025-01-27 16:43:32,357: INFO: 3420280441: Evaluation instance created successfully.]
[2025-01-27 16:43:32,358: INFO: 3420280441: Starting model evaluation.]
[2025-01-27 16:43:32,360: INFO: 3420280441: Loading model from: artifacts/training/trained_model.keras]
[2025-01-27 16:43:34,550: INFO: 3420280441: Model loaded successfully.]
[2025-01-27 16:43:34,552: INFO: 3420280441: Initializing the validation data generator.]
[2025-01-27 16:43:34,555: ERROR: 3420280441: Error initializing validation generator: [Errno 2] No such file or directory: 'data/train']
[2025-01-27 16:43:34,557: ERROR: 3420280441: Error during evaluation: [Errno 2] No such file or directory: 'data/train']


FileNotFoundError: [Errno 2] No such file or directory: 'data/train'